# 1.  Learn-One-Rule函数

Learn-One-Rule函数的目标是**提取一个分类规则，该规则覆盖训练集中的大量正例，没有或仅覆盖少量反例。**

然而，由于搜索空间呈指数大小，要找到一个最佳的规则的计算开销很大。

Learn-One-Rule函数通过以一种贪心的方式的增长规则来解决指数搜索问题。

它产生一个初始规则$r$，并不断对该规则求精，直到满足某种终止条件为止。然后修剪该规则，以改进它的泛化误差。

**规则增长策略**：

常见的分类规则增长策略有两种：

从**一般到特殊**和**从特殊到一般。**

>在从一般到特殊的策略中，先建立一个初始规则$r:\{\}→y$，其中左边是一个空集，右边包含目标类。

该规则的质量很差，因为它覆盖训练集中的所有样例。接着加入新的合取项来提高规则的质量，直到满足终止条件为止（例如，加入的合取项已不能提高规则的质量）。

>对于从特殊到一般的策略，可以随机地选择一个正例作为规则增长的初始种子。

再求精步，通过删除规则的一个合取项，使其覆盖更多的正例来范化规则。重复求精步，直到满足终止条件为止（例如，当规则开始覆盖反例时为止）。

由于规则的贪心的方式增长，以上方法可能会产生次优规则。

为了避免这种问题，可以采用束状搜索（beam search）。

算法维护$k$个最佳候选规则，各候选规则各自在其前件中添加或删除合取项而独立地增长。

评估候选规则的质量，选出$k$个最佳候选进入下一轮迭代。

**规则评估：**

在规则的增长过程中，**需要一种评估度量来确定应该添加（或删除）哪个合取项**。

准确率就是一个很明显的选择，因为它明确地给出了被规则正确分类的训练样例的比例。**然而把准确率作为标准的一个潜在的局限性是它没有考虑规则的覆盖率。**

下面的方法可以用来处理该问题：

（1）**可以使用统计检验剪除覆盖率较低的规则。**

例如，我们可以计算下面的似然比（likelihood ratio）统计量：

$$R=2\sum_{i=1}^mf_ilog(\frac{f_i}{e_i})$$

其中，$m$是类的个数，$f_i$ 是被规则覆盖的类$i$的样本的观测频率，$e_i$ 是规则作随机猜想的期望频率。注意R是满足自由度为$m-1$的$χ^2$分布。较大的R值说明该规则做出的正确预测数显著地大于随机猜测的结果。

（2）**可以使用一种考虑规则覆盖率的评估度量。**

考虑如下评估度量：
$$Laplace = \frac{f_++1}{n+k}$$

$$m估计=\frac{f_++kp_+}{n+k}$$

其中n是规则覆盖的样例数，f+是规则覆盖的正例数，k是类的总数，p+是正类的先验概率。注意当p+=1/k时，m估计等价于Laplace度量。

（3）**另一种可以使用的评估度量是考虑规则的支持度计数的评估度量。**

FOIL信息增益就是一种这样的度量。规则的支持度计数对应于它所覆盖的正例数。

假设规则$r : A→+$覆盖$p_0$个正例和$n_0$个反例。增加新的合取项B，扩展后的规则$r' : A∧B→+$覆盖$p_1$个正例和$n_1$个反例。

根据以上信息，扩展后规则的FOIL信息增益定义为：

$$FOIL信息增益=p_1*(log_2\frac{p_1}{p_1+n_1}-log_2\frac{p_0}{p_0+n_0})$$

由于该度量与$p1$和$p1/p1+n1$成正比，因此它更倾向于选择那些高支持度计数和高准确率的规则。

规则减枝 

可以对Learn-One-Rule函数产生的规则进行减枝，以改善它们的泛化误差。

# 2. 顺序覆盖基本原理


规则提取出来后，顺序覆盖算法必须删除该规则所覆盖的所有正例和反例。


# 3. RIPPER算法

为了阐明规则提取的直接方法，考虑一种广泛使用的规则归纳算法，叫作RIPPER算法。

该算法的复杂度几乎线性地随训练样例的数目增长，并且特别适合为类分布不平衡的数据集建立模型。

RIPPER也能很好地处理噪声数据集，因为它使用一个确认数据集来防止模型过分拟合。

对两类问题，RIPPER算法选择以多数类作为默认类，并为预测少数类学习规则。

对于多类问题，先按类的频率对类进行排序，设$（y1,y2,…,yc）$是排序后的类，其中$y_1$是最不频繁的类，$y_c$是最频繁的类。

第一次迭代中，把属于$y_1$的样例标记为正例，而把其他类的样例标记为反例，使用顺序覆盖算法产生区分正例和反例的规则。

接下来，RIPPER提取区分$y_2$和其他类的规则。重复该过程，直到剩下类$y_c$，此时$y_c$作为默认类。

**规则增长**

RIPPER算法使用从一般到特殊的策略进行规则增长，使用FOIL信息增益来选择最佳合取项添加到规则前件中。

当规则开始覆盖反例时，停止添加合取项。

新规则根据其在确认集上的性能进行减枝。

计算下面的度量来确定规则是否需要减枝：$(p-n)/(p+n)$，其中p和n分别是被规则覆盖的确认集中的正例和反例数目，关于规则在确认集上的准确率，该度量是单调的。

如果减枝后该度量增加，那么就去掉该合取项。减枝是从最后添加的合取项开始的。

例如给定规则$ABCD→y$，RIPPER算法先检查D是否应该减枝，然后是CD、BCD等。

尽管原来的规则仅覆盖正例，但是减枝后的规则可能会覆盖训练集中的一些反例。

**建立规则集** 

规则生成后，它所覆盖的所有正例和反例都要被删除。只要该规则不违反基于最小描述长度的终止条件，就把它添加到规则集中。

如果新规则把规则集的总描述长度增加了至少d个比特位，那么RIPPER就停止把该规则加入到规则集（默认的d是64位）。

RIPPER使用的另一个终止条件是规则在确认集上的错误率不超过50%。

RIPPER算法也采用其他的优化步骤来决定规则集中现存的某些规则能否被更好的规则替代。

# 规则提取的间接方法

原则上，决策树从根节点到叶节点的每一条路径都可以表示为一个分类规则。

路径中的测试条件构成规则前件的合取项，叶节点的类标号赋给规则后件。

注意，规则集是完全的，包含的规则是互斥的。

下面，介绍C4.5规则算法所采用的从决策树生成规则集的方法。

**规则产生**

决策树中从根节点到叶节点的每一条路径都产生一条分类规则。

给定一个分类规则$r : A→y$，考虑简化后的规则 $r': A' →y$，其中A'是从A去掉一个合取项后得到的。

只要简化后的规则的误差率低于原规则的误差率，就保留其中悲观误差率最低的规则。

重复规则减枝步骤，直到规则的悲观误差不能再改进为止。由于某些规则在减枝后会变得相同，因此必须丢弃重复规则。

**规则排序** 

产生规则集后，C4.5规则算法使用基于类的排序方案对提取的规则定序。

预测同一个类的规则分到同一个子集中。计算每个子集的总描述长度，然后各类按照总描述长度由小到大排序。

具有最小描述长度的类优先级最高，因为期望它包含最好的规则集。类的总描述长度等于Lexception+g×Lmodel，其中Lexception是对误分类样例编码所需的比特位数，Lmodel是对模型编码所需要的比特位数，而g是调节参数，默认值为0.5,。

调节参数的值取决于模型中冗余属性的数量，如果模型含有很多冗余属性，那么调节参数的值会很小。

# 基于规则的分类器的特征

基于规则的分类器有如下特点：

>规则集的表达能力几乎等价于决策树，因为决策树可以用互斥和穷举的规则集表示。

>基于规则分类器和决策树分类器都对属性空间进行直线划分，并将类指派到每个划分。

>然而，如果基于规则的分类器允许一条记录触发多条规则的话，就可以构造一个更加复杂的决策边界。

>基于规则的分类器通常被用来产生更易于解释的描述性模型，而模型的性能却可与决策树分类器相媲美。

>被很多基于规则的分类器（如RIPPER）所采用的基于类的规则定序方法非常适于处理类分布不平衡的数据集。